In [ ]:
!pip install torch torchvision
!pip install opencv-python-headless
!pip install recognition
!pip install tqdm
!pip install -q torch torchvision
!pip install -q opencv-python-headless dlib face_recognition
!pip install face_recognition
!pip install dlib

ERROR: Could not find a version that satisfies the requirement recognition (from versions: none)
ERROR: No matching distribution found for recognition
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
!pip install face_recognition
!pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=0c2423dda4bb47663a47a236d5591adb274681f5f1ab711e193232509bf1382c
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


In [ ]:
# Veri indirme
from google.colab import files
files.upload()

# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Gerekli kütüphaneleri kurulumu
!pip install -q kaggle kagglehub

# Kaggle API anahtarını tekrar doğru yere taşıma
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/json

# Google Drive'da verileri koyacağımız klasörü oluşturalım (varsa hata vermez)
!mkdir -p /content/drive/MyDrive/Bitcoin Price Prediction Project/datasets

# VERİYİ İNDİR VE DOĞRUDAN GOOGLE DRIVE'A YÜKLE
import kagglehub

# Veri setini tekrar Colab'ın geçici hafızasına indirelim
print("KaggleHub ile veri seti indirme işlemi başladı")
path = kagglehub.dataset_download("xdxd003/ff-c23")
print(f"Veri seti başarıyla şu geçici yola indirildi: {path}")

# ZIP dosyasını BU SEFER DOĞRUDAN Google Drive'daki hedef klasörümüze çıkartalım
print("ZIP dosyası DOĞRUDAN Google Drive'a çıkartılıyor...")

!unzip -q "{path}"/*.zip -d /content/drive/MyDrive/Bitcoin Price Prediction Project/datasets/

print("Veri seti başarıyla Google Drive'a çıkartıldı!")

In [ ]:
# VİZE İÇİN BASE MODEL VERİ İŞLEME (250 REAL + 250 FAKE)

import os
import cv2
import face_recognition
from google.colab import drive
from tqdm.notebook import tqdm
import glob
import random

# Drive'ı bağla ve yolları tanımla
drive.mount('/content/drive')

DRIVE_BASE_PATH = "/content/drive/MyDrive/Bitcoin Price Prediction Project"
RAW_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "datasets", "FaceForensics++_C23")
# Çıktı klasörü
OUTPUT_PATH = os.path.join(DRIVE_BASE_PATH, "processed_data_500_videos")

NUM_FRAMES_PER_VIDEO = 20 # Her videodan 20 kare alalım
IMAGE_SIZE = 224

# Çıktı klasörlerini oluşturma
os.makedirs(os.path.join(OUTPUT_PATH, "real"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "fake"), exist_ok=True)

print(f"Ham video veriler: {RAW_DATA_PATH}")
print(f"İşlenmiş yüz resimleri buraya kaydedilecek: {OUTPUT_PATH}")

def process_videos_final(video_paths, output_dir_label):

    output_path = os.path.join(OUTPUT_PATH, output_dir_label)

    for video_path in tqdm(video_paths, desc=f"Processing {output_dir_label} videos"):
        video_name = os.path.basename(video_path).split('.')[0]
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened(): continue
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames < 1: continue

            frame_indices = [int(i) for i in (total_frames / NUM_FRAMES_PER_VIDEO * (j + 0.5) for j in range(NUM_FRAMES_PER_VIDEO))] if total_frames > NUM_FRAMES_PER_VIDEO else list(range(total_frames))

            frame_count = 0
            saved_frames = 0
            while saved_frames < len(frame_indices):
                ret, frame = cap.read()
                if not ret: break
                if frame_count in frame_indices:
                    face_locations = face_recognition.face_locations(frame)
                    if len(face_locations) > 0:
                        top, right, bottom, left = face_locations[0]
                        face_image = frame[top:bottom, left:right]
                        if face_image.size > 0:
                            resized_face = cv2.resize(face_image, (IMAGE_SIZE, IMAGE_SIZE))
                            save_name = f"{video_name}_frame_{frame_count}.jpg"
                            cv2.imwrite(os.path.join(output_path, save_name), resized_face)
                            saved_frames += 1
                frame_count += 1
            cap.release()
        except Exception as e:
            print(f"Bir hata oluştu: {video_name} - {e}")
            continue

# VERİ SEÇİMİ VE OPERASYON BAŞLANGICI

# Adım 1: 250 Gerçek video seçimi
real_videos = sorted(glob.glob(os.path.join(RAW_DATA_PATH, "original", "*.mp4")))[:250]

# Adım 2: Her bir fake klasöründen 50'şer video seçimi
fake_video_folders = ["Deepfakes", "Face2Face", "FaceShifter", "FaceSwap", "NeuralTextures"]
fake_videos = []
for folder in fake_video_folders:
    videos_in_folder = sorted(glob.glob(os.path.join(RAW_DATA_PATH, folder, "*.mp4")))
    fake_videos.extend(videos_in_folder[:50])

print(f"İşlemek için {len(real_videos)} gerçek video seçildi.")
print(f"İşlemek için {len(fake_videos)} sahte video seçildi (her bir 5 teknikten 50'şer adet).")

# Operasyon
process_videos_final(real_videos, "real")
process_videos_final(fake_videos, "fake")

print("\n\n BASE MODEL İÇİN VERİ İŞLEME TAMAMLANDI!")
print(f"Tüm yüz resimleri Google Drive'daki '{OUTPUT_PATH}' klasörüne kaydedildi.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# MODEL TANIMI: Batch Normalization Model
# Farklar: 4 Katmanlı, Batch Norm var, Leaky ReLU kullanılıyor.

class BatchNormCNN(nn.Module):
    def __init__(self):
        super(BatchNormCNN, self).__init__()

        # 1. Katman Bloğu
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32) # Batch Norm
        self.pool = nn.MaxPool2d(2, 2)

        # 2. Katman Bloğu
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # 3. Katman Bloğu
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # 4. Katman Bloğu
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        # Tam Bağlantı Katmanları
        # 224 -> pool -> 112 -> pool -> 56 -> pool -> 28 -> pool -> 14
        # Çıkış boyutu: 256 kanal * 14 * 14
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.dropout = nn.Dropout(0.6)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        # Leaky ReLU kullanıyoruz (Negatif değerleri hafifçe geçirir)
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)), 0.1))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x)), 0.1))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x)), 0.1))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x)), 0.1))

        x = x.view(-1, 256 * 14 * 14) # Flatten
        x = F.leaky_relu(self.fc1(x), 0.1)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

print("Model (BatchNormCNN) başarıyla oluşturuldu")

import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
import cv2
from PIL import Image
import glob
from tqdm.notebook import tqdm

# AYARLAR
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAVE_PATH = "/content/drive/MyDrive/Bitcoin Price Prediction Project/checkpoints/muhammed_best_model.pth"
DATA_PATH = "/content/drive/MyDrive/Bitcoin Price Prediction Project/processed_data_500_videos"

# Dataset Sınıfı (Aynısı)
class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        real_path = os.path.join(root_dir, "real")
        fake_path = os.path.join(root_dir, "fake")

        for img in glob.glob(os.path.join(real_path, "*.jpg")):
            self.image_paths.append(img)
            self.labels.append(0.0) # REAL = 0

        for img in glob.glob(os.path.join(fake_path, "*.jpg")):
            self.image_paths.append(img)
            self.labels.append(1.0) # FAKE = 1

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label).float()

# Veri Hazırlığı
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = DeepfakeDataset(DATA_PATH, transform=transform)

# Train/Val Split (%80 - %20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# MODELİ BAŞLAT
model = BatchNormCNN().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Eğitim başlıyor... Cihaz: {DEVICE}")

best_val_acc = 0.0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Training"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.sigmoid(outputs) > 0.5
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    # Validation Kısmı
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    train_acc = 100 * correct_train / total_train
    val_acc = 100 * correct_val / total_val

    print(f"Epoch {epoch+1} -> Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"--> Yeni en iyi model kaydedildi! Acc: {val_acc:.2f}%")

print("Eğitim Tamamlandı!")

Model (BatchNormCNN) başarıyla oluşturuldu
Eğitim başlıyor... Cihaz: cpu


Epoch 1/10 Training:   0%|          | 0/175 [00:00<?, ?it/s]

Epoch 1 -> Loss: 2.0974 | Train Acc: 64.02% | Val Acc: 70.75%
--> Yeni en iyi model kaydedildi! Acc: 70.75%


Epoch 2/10 Training:   0%|          | 0/175 [00:00<?, ?it/s]

Epoch 2 -> Loss: 0.7431 | Train Acc: 67.87% | Val Acc: 75.41%
--> Yeni en iyi model kaydedildi! Acc: 75.41%


...

Epoch 8 -> Loss: 0.4293 | Train Acc: 79.46% | Val Acc: 81.29%


Epoch 9/10 Training:   0%|          | 0/175 [00:00<?, ?it/s]

Epoch 9 -> Loss: 0.4355 | Train Acc: 79.63% | Val Acc: 81.15%


Epoch 10/10 Training:   0%|          | 0/175 [00:00<?, ?it/s]

Epoch 10 -> Loss: 0.4165 | Train Acc: 80.51% | Val Acc: 80.36%
Eğitim Tamamlandı!


In [ ]:
import os
import glob
import shutil

# Proje Yolları
DRIVE_BASE_PATH = "/content/drive/MyDrive/Bitcoin Price Prediction Project"
RAW_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "datasets", "FaceForensics++_C23")
TEST_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "final_test_data_25_25") # Anlaşılır olması için yeni bir isim verdim

# Önceki test verisi varsa diye temizleyelim
if os.path.exists(TEST_DATA_PATH):
    shutil.rmtree(TEST_DATA_PATH)
    print(f"Eski '{TEST_DATA_PATH}' klasörü temizlendi.")

# Test klasörlerini yeniden oluşturalım
os.makedirs(os.path.join(TEST_DATA_PATH, "real"), exist_ok=True)
os.makedirs(os.path.join(TEST_DATA_PATH, "fake"), exist_ok=True)
print(f"Yeni '{TEST_DATA_PATH}' klasörü oluşturuldu.")

# ==============================================================================
# 1. 25 ADET GERÇEK VİDEO AL
# ==============================================================================
# Eğitimde ilk 250'yi kullanmıştık. Şimdi 250'den sonraki 25 videoyu alıyoruz.
real_videos_to_test = sorted(glob.glob(os.path.join(RAW_DATA_PATH, "original", "*.mp4")))[250:275]

for video in real_videos_to_test:
    shutil.copy(video, os.path.join(TEST_DATA_PATH, "real"))

print(f"-> {len(real_videos_to_test)} adet GERÇEK video 'real' klasörüne başarıyla kopyalandı.")


# ==============================================================================
# 2. 25 ADET SAHTE VİDEO AL (SADECE DEEPFAKES KLASÖRÜNDEN)
# ==============================================================================
# Eğitimde ilk 50'yi kullanmıştık. Şimdi 50'den sonraki 25 videoyu alıyoruz.
fake_videos_to_test = sorted(glob.glob(os.path.join(RAW_DATA_PATH, "Deepfakes", "*.mp4")))[50:75]

for video in fake_videos_to_test:
    shutil.copy(video, os.path.join(TEST_DATA_PATH, "fake"))

print(f"-> {len(fake_videos_to_test)} adet SAHTE video 'fake' klasörüne başarıyla kopyalandı.")

print("\n25 gerçek ve 25 sahte videodan oluşan test seti hazır!")

In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os, glob, cv2, face_recognition
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from google.colab import drive

drive.mount('/content/drive')

# --- AYARLAR ---
# Arkadaşının modelinin kaydedildiği yol
MODEL_PATH = "/content/drive/MyDrive/Bitcoin Price Prediction Project/checkpoints/muhammed_best_model.pth"
TEST_DATA_PATH = "/content/drive/MyDrive/Bitcoin Price Prediction Project/final_test_data_25_25" # Senin oluşturduğun test seti
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- MODEL CLASS TEKRARI (Notebook'ta yukarıda tanımlıysa burayı silmene gerek yok ama garanti olsun) ---
class BatchNormCNN(nn.Module):
    def __init__(self):
        super(BatchNormCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # 4. Katman
        self.bn4 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.dropout = nn.Dropout(0.6)
        self.fc2 = nn.Linear(512, 1)
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)), 0.1))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x)), 0.1))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x)), 0.1))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x)), 0.1))
        x = x.view(-1, 256 * 14 * 14)
        x = F.leaky_relu(self.fc1(x), 0.1)
        x = self.dropout(x); x = self.fc2(x)
        return x

# --- MODELİ YÜKLE ---
print("Arkadaşının modeli (BatchNormCNN) yükleniyor...")
model = BatchNormCNN().to(device)
try:
    model.load_state_dict(torch.load(MODEL_PATH))
    print("Model başarıyla yüklendi.")
except:
    print("Model dosyası bulunamadı! Önce eğitimi çalıştır.")

model.eval()

# --- PREDICTION FONKSİYONU ---
def predict_video(video_path, model):
    transform = transforms.Compose([
        transforms.Resize((224, 224)), transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]) # Dikkat: Eğitimdeki Normalize ile aynı olmalı

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): return "HATA"

    face_predictions = []
    frame_count = 0

    while frame_count < 30:
        ret, frame = cap.read()
        if not ret: break

        face_locations = face_recognition.face_locations(frame)
        if len(face_locations) > 0:
            top, right, bottom, left = face_locations[0]
            face_image = frame[top:bottom, left:right]
            if face_image.size > 0:
                pil_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
                input_tensor = transform(pil_image).unsqueeze(0).to(device)
                with torch.no_grad():
                    output = model(input_tensor)
                    prob = torch.sigmoid(output).item()
                    face_predictions.append(1 if prob > 0.5 else 0)
        frame_count += 1
    cap.release()

    if not face_predictions: return "Yüz Bulunamadı"
    # Majority Voting
    final_prediction = 1 if sum(face_predictions) > len(face_predictions) / 2 else 0
    return "FAKE" if final_prediction == 1 else "REAL"

# --- TEST DÖNGÜSÜ ---
all_videos = glob.glob(os.path.join(TEST_DATA_PATH, "**", "*.mp4"), recursive=True)
true_labels, predicted_labels = [], []

print("Test başlıyor...")
for video_path in tqdm(all_videos, desc="Test Videoları"):
    ground_truth = "REAL" if "real" in video_path else "FAKE"
    prediction = predict_video(video_path, model)

    if prediction in ["REAL", "FAKE"]:
        true_labels.append(ground_truth)
        predicted_labels.append(prediction)

# --- SONUÇLARI YAZDIR ---
acc = accuracy_score(true_labels, predicted_labels)
prec = precision_score(true_labels, predicted_labels, pos_label='FAKE')
rec = recall_score(true_labels, predicted_labels, pos_label='FAKE')
f1 = (2 * prec * rec) / (prec + rec) if (prec + rec) > 0 else 0
cm = confusion_matrix(true_labels, predicted_labels, labels=["REAL", "FAKE"])

print("\n" + "="*40)
print(f" BatchNormCNN SONUÇLARI")
print("="*40)
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")
print("-" * 20)
print("Confusion Matrix:")
print(cm)